In [1]:
!pip install pandas scikit-learn torch transformers

In [1]:
# Load and process data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import Trainer, TrainingArguments

# Load the CSV file
data = pd.read_csv("Hypertension-risk-model-main.csv")
data = data.dropna()
data = data[data['Risk'].isin([0, 1])]

# Display the first few rows of the data
data.head()

/Users/asujithan/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,male,age,currentSmoker,cigsPerDay,BPMeds,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,Risk
0,1,39,0,0.0,0.0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0.0,0.0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1,20.0,0.0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,30.0,0.0,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,23.0,0.0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
from torch import nn

In [3]:
# model = torch.nn.Sequential(
#     nn.Linear(12, 64),
#     nn.Linear(64, 56),
#     nn.Linear(56, 48),
#     nn.Linear(48, 32),
#     nn.Linear(32, 1),
#     nn.SiLU(),
#     nn.ReLU(),
#     nn.Softmax(dim=12),
#     nn.Sigmoid()
# )

class HypertensionRiskModel(nn.Module):
    def __init__(self, input_dim):
        super(HypertensionRiskModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 56)
        self.fc5 = nn.Linear(56, 48)
        self.fc6 = nn.Linear(48, 32)
        self.fc7 = nn.Linear(32, 1)
        self.silu = nn.SiLU()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.silu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.silu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        x = self.silu(x)
        x = self.fc6(x)
        x = self.silu(x)
        x = self.fc7(x)
        x = self.sigmoid(x)  # For binary classification, sigmoid at the end
        return x
    

model = HypertensionRiskModel(12)

In [4]:
# Define features and target
X = data.drop(columns=["Risk"])
y = data["Risk"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [5]:
# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/50], Loss: 0.3049
Epoch [2/50], Loss: 0.2636
Epoch [3/50], Loss: 0.2582
Epoch [4/50], Loss: 0.2519
Epoch [5/50], Loss: 0.2477
Epoch [6/50], Loss: 0.2444
Epoch [7/50], Loss: 0.2429
Epoch [8/50], Loss: 0.2354
Epoch [9/50], Loss: 0.2335
Epoch [10/50], Loss: 0.2312
Epoch [11/50], Loss: 0.2271
Epoch [12/50], Loss: 0.2274
Epoch [13/50], Loss: 0.2214
Epoch [14/50], Loss: 0.2207
Epoch [15/50], Loss: 0.2182
Epoch [16/50], Loss: 0.2122
Epoch [17/50], Loss: 0.2071
Epoch [18/50], Loss: 0.2037
Epoch [19/50], Loss: 0.1978
Epoch [20/50], Loss: 0.1958
Epoch [21/50], Loss: 0.1870
Epoch [22/50], Loss: 0.1833
Epoch [23/50], Loss: 0.1793
Epoch [24/50], Loss: 0.1699
Epoch [25/50], Loss: 0.1659
Epoch [26/50], Loss: 0.1574
Epoch [27/50], Loss: 0.1509
Epoch [28/50], Loss: 0.1455
Epoch [29/50], Loss: 0.1461
Epoch [30/50], Loss: 0.1314
Epoch [31/50], Loss: 0.1343
Epoch [32/50], Loss: 0.1183
Epoch [33/50], Loss: 0.1211
Epoch [34/50], Loss: 0.1026
Epoch [35/50], Loss: 0.1112
Epoch [36/50], Loss: 0.1048
E

In [6]:
# Evaluation function
def evaluate_model(loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for X_batch, y_batch in loader:
            outputs = model(X_batch)
            predicted = (outputs > 0.5).float()
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Evaluate on the test set
test_accuracy = evaluate_model(test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 86.44%


In [17]:
# Example prediction
example = torch.tensor([[0, 45, 1, 30, 0, 0, 200, 140, 90, 25, 80, 110]], dtype=torch.float32)
example = scaler.transform(example)  # Scale the example
example = torch.tensor(example, dtype=torch.float32)  # Convert to tensor

with torch.no_grad():
    prediction = model(example)
    risk = "at risk" if prediction > 0.5 else "not at risk"
    print(f"The individual is {risk}.")

The individual is not at risk.


/Users/asujithan/anaconda3/envs/pytorch/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [18]:
torch.save(model.state_dict(), "hypertension_risk_model.pth")